In [9]:
from src.eda.soporte_eda import *

In [2]:
conectar_bd()

(<connection object at 0x0000015C9A2F5AD0; dsn: 'user=postgres password=xxx dbname=Proyecto_Sephora host=localhost port=5432', closed: 0>,
 <cursor object at 0x0000015C9A29F680; closed: 0>)

En principio se parte de esta query general con los aspectos más relevantes de los productos (aspectos como numero de variaciones o el acabado, textura etc. son caracteristicas consideradas mas secundarias y menos relevantes de los productos que se usaran en el analisis mas especifico para determinar el producto a lanzar pero en el analisis general inicial de Sephora no se consideran). este analisis general inicial tiene como proposito posicionar la marca de sephora collection dentro de sephora, es decir, en comparacion con las otras marcas

In [3]:
query_general = """
select 
    p.url_producto,
    p.nombre,
    p.descripcion,
    m.nombre_marca, 
    c.nombre_categoria,
    s.nombre_subcategoria,
    h.precio,
    h.numero_valoraciones,
    h.valoracion
from productos p 
join marcas m on p.id_marca = m.id_marca
left join categorias c on p.id_categoria = c.id_categoria
left join subcategorias s on p.id_subcategoria = s.id_subcategoria
join historico h on p.id_producto = h.id_producto;
"""

In [58]:
df = ejecutar_query(query_general)
df.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion
0,https://www.sephora.es/p/soft-lit-naturally-lu...,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7
1,https://www.sephora.es/p/triclone-skin-tech-fo...,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5
2,https://www.sephora.es/p/triclone-skin-tech-hy...,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1
3,https://www.sephora.es/p/color-fuse-talc-free-...,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8
4,https://www.sephora.es/p/bio-radiant-gel-powde...,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8


In [6]:
df.shape

(1829, 9)

In [11]:
def info_reporte(dataframe):
    """
    Genera un informe sobre los valores nulos y tipos de datos de un DataFrame.

    Args:
        dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
        pd.DataFrame: Un DataFrame con el número y porcentaje de valores nulos por columna,
                      junto con el tipo de dato de cada columna.
    """
    df_report = pd.DataFrame()
    df_report["Numero_nulos"] = dataframe.isnull().sum()
    df_report["Porcentaje_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0]*100), 2)
    df_report["Tipo_dato"] = dataframe.dtypes
    return df_report

In [12]:
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
nombre,3,0.16,object
descripcion,98,5.36,object
nombre_marca,0,0.00,object
nombre_categoria,9,0.49,object
nombre_subcategoria,33,1.80,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64


In [13]:
df["url_producto"].duplicated().sum()

np.int64(7)

In [36]:
df.drop_duplicates(subset=["url_producto"], inplace=True)
df.shape

(1822, 9)

In [24]:
df.head()

,url_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion
0,https://www.sephora.es/p/soft-lit-naturally-lu...,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7
1,https://www.sephora.es/p/triclone-skin-tech-fo...,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5
2,https://www.sephora.es/p/triclone-skin-tech-hy...,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1
3,https://www.sephora.es/p/color-fuse-talc-free-...,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8
4,https://www.sephora.es/p/bio-radiant-gel-powde...,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8


Me quito los duplicados (comento el problema). y explico que lo hago por el campo de url porque me he dado cuenta de que por los campos principales que habia de los productos en la tabla de productos pues hay productos que si que estan duplicados en nombre, descripcion, marca, categoria y subcategoria aunque verdaderamente no sean los mismos (puedo demostrarlo tambien)

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,1822.0,37.201850,18.895532,3.99,25.0,34.99,45.00,169.0
numero_valoraciones,1822.0,590.726674,1707.459476,0.00,3.0,42.50,406.75,20035.0
valoracion,1822.0,3.799671,1.543679,0.00,3.9,4.40,4.70,5.0


Aqui tengo que hacer un buen analisis de maximos / minimos (outliers). aqui es donde puedo meter algun grafico. poder quizas identificar si esos productos mas outliers se corresponden con alguna marca / categoria o subcategoria concreta o no, y en el caso de las valoraciones con precios mas altos o bajos y tal 

In [16]:
df.describe(include="object").T

,count,unique,top,freq
url_producto,1822,1822,https://www.sephora.es/p-640056.html,1
nombre,1819,1646,Moving Lights,14
descripcion,1724,1204,Paleta de sombras de ojos,44
nombre_marca,1822,79,Sephora Collection,198
nombre_categoria,1813,13,Rostro,627
nombre_subcategoria,1789,48,Máscara de pestañas,148


Recuento de productos por marca

In [37]:
df_result = df.groupby("nombre_marca")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
df_result.head(10)

,nombre_marca,url_producto
63,Sephora Collection,198
5,Benefit Cosmetics,86
10,Charlotte Tilbury,77
42,Make Up For Ever,66
15,Dior,65
69,Tarte,64
0,Anastasia Beverly Hills,61
66,Sisley,58
21,Fenty Beauty,58
39,Lancôme,57


Recuento de productos por categoria 

In [39]:
df_result = df.groupby("nombre_categoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
df_result.head(10)

,nombre_categoria,url_producto
10,Rostro,627
4,Ojos,372
3,Labios,343
8,Pinceles & accesorios,167
0,Cejas,100
1,Desmaquillante,58
5,Paletas maquillaje,49
12,Uñas,43
2,Estuches maquillaje,42
11,Sacapuntas,4


Recuento de procuctos por subcategoria 

In [40]:
df_result = df.groupby("nombre_subcategoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
df_result.head(10)

,nombre_subcategoria,url_producto
32,Máscara de pestañas,148
2,Barra de labios,126
8,Colorete,113
4,Base de maquillaje,107
42,Primer & spray fijador,104
6,Brocha rostro,103
1,Antiojeras & corrector,80
25,Gloss/ Brillo labial,79
44,Sombras de ojos,76
34,Paleta de sombras de ojos,74


Productos considerados ya bastante exitosos dentro de Sephora (me baso sobretodo en las medianas aunque redondeando un poco los valores al alza, es decir, aproximándolo incluso un poco más por encima del 50% de la distribución. decido marcar los limites en valoracion de 4,5 que ya considero que es bastante buena valoracion y ademas el percentil 75 esta en 4,7, es decir que no se aleja casi de la valoracion que tan solo logran tener mas del 25% de los productos. en el caso del numero de valoraciones basandome tambien un poco en la mediana aunque redondeando al alza he fijado el limite en 50 valoraciones pues considero que conocer que tan solo el 50% de los productos tiene mas de 42 valoraciones ya es indicador de que de por si aquellos productos que tienen mas valoraciones tampoco tienen demasiadas con excepcion de aquellos productos "outliers" que si cuentan con muchas valoraciones. y he escogido la mediana tan bien mas bien que el percentil 75 porque al final aqui estoy combinando ambos aspectos, el numero de valoraciones y la valoracion por lo que considero que ya de por si es bastante buen indicador que combine lo que por separado tan solo consiguen menos del 50% de los productos)

In [57]:
exitosos = df[(df["valoracion"] >= 4.5) & (df["numero_valoraciones"] >= 50)]

print("Número de productos exitosos:", len(exitosos))

Número de productos exitosos: 393


aqui indicar quizas que % suponen sobre el total de productos de sephora 

Productos exitosos por marca 

In [53]:
exitosos_por_marca = exitosos.groupby("nombre_marca")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
exitosos_por_marca.head(10)

,nombre_marca,url_producto
3,Benefit Cosmetics,35
37,Sephora Collection,32
30,Natasha Denona,29
0,Anastasia Beverly Hills,27
19,Huda Beauty,23
25,Make Up For Ever,21
23,Lancôme,18
29,Nars,18
18,Hourglass,14
5,Charlotte Tilbury,12


Productos exitosos por categorias

In [54]:
exitosos_por_categoria = exitosos.groupby("nombre_categoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
exitosos_por_categoria.head(10)

,nombre_categoria,url_producto
7,Rostro,152
3,Labios,69
4,Ojos,63
6,Pinceles & accesorios,46
0,Cejas,26
5,Paletas maquillaje,20
1,Desmaquillante,10
9,Uñas,5
2,Estuches maquillaje,1
8,Sacapuntas,1


Productos exitosos por subcategorias

In [55]:
exitosos_por_subcategoria = exitosos.groupby("nombre_subcategoria")["url_producto"].count().reset_index().sort_values("url_producto", ascending=False)
exitosos_por_subcategoria.head(10)

,nombre_subcategoria,url_producto
8,Colorete,43
6,Brocha rostro,30
25,Paleta de sombras de ojos,29
2,Barra de labios,28
33,Sombras de ojos,23
19,Iluminador,19
24,Máscara de pestañas,18
4,Base de maquillaje,16
28,Polvos bronceadores,15
21,Lápiz & polvo para cejas,14


en estas tablas de group by lo que puedo hacer es añadir una columna que sea % sobre el total (en las de arriba sobre el total de productos general de sephora y en estas de productos exitosos quizas mejor sobre el total de productos que tienen exito en sephora)

observar si estos productos mas exitosos estan correspondidos con precios mas altos o mas bajos y puede haber ahi cierta relacion

In [60]:
precio_medio_exitosos = exitosos["precio"].mean()
precio_medio_exitosos

np.float64(38.9875572519084)

Al ser el precio medio casi el mismo que de los productos de Sephora en general (Incluso el de los exitosos un pelin mas elevado), sugiere que precios mas bajos no son lo que justifican principalmente a los productos mas exitosos. influiran otros factores

precios medios por marcas 

In [62]:
precio_medio_marca = df.groupby("nombre_marca")["precio"].mean().reset_index().sort_values("precio", ascending=False)
precio_medio_marca.tail(10)

,nombre_marca,precio
9,Caudalie,19.666667
3,BEAUTY OF JOSEON,18.000000
26,Glow Recipe,17.500000
7,Byoma,16.500000
41,Lanolips,15.937500
63,Sephora Collection,15.846061
77,Youth To The People,15.000000
70,The Ordinary,14.990000
61,SVR,13.333333
46,Merci Handy,12.990000


precios medios por categorias

In [63]:
precio_medio_categoria = df.groupby("nombre_categoria")["precio"].mean().reset_index().sort_values("precio", ascending=False)
precio_medio_categoria.head(10)

,nombre_categoria,precio
5,Paletas maquillaje,61.396400
10,Rostro,42.507528
1,Desmaquillante,38.136552
2,Estuches maquillaje,36.521905
4,Ojos,35.445457
8,Pinceles & accesorios,34.103750
3,Labios,32.021134
0,Cejas,31.107400
7,Pestañas postizas,22.250000
11,Sacapuntas,20.745000


precios medios por subcategorias

In [64]:
precio_medio_subcategoria = df.groupby("nombre_subcategoria")["precio"].mean().reset_index().sort_values("precio", ascending=False)
precio_medio_subcategoria.head(10)

,nombre_subcategoria,precio
36,Paleta rostro,59.664000
34,Paleta de sombras de ojos,59.517333
33,Paleta de contouring,55.000000
4,Base de maquillaje,54.913645
35,Paleta multi-usos,51.000000
40,Polvos matificantes,46.499063
18,Estuches de manicura,45.396000
10,Crema de color,44.097000
39,Polvos bronceadores,44.017885
12,Desmaquillante rostro,43.256000


In [67]:
precio_medio_marca_subcat = (
    df.groupby(["nombre_categoria", "nombre_marca"])["precio"]
    .mean()
    .reset_index()
    .sort_values(by=["nombre_categoria", "precio"], ascending=[True, False])
)
precio_medio_marca_subcat

,nombre_categoria,nombre_marca,precio
24,Cejas,Sisley,60.000000
9,Cejas,Grande Cosmetics,54.990000
27,Cejas,Valentino,43.000000
11,Cejas,Hourglass,37.000000
5,Cejas,Dior,35.666667
...,...,...,...
274,Sacapuntas,Sephora Collection,3.990000
275,Uñas,Dior,35.666667
276,Uñas,Isdin,27.000000
277,Uñas,Manucurist,17.707143


esto tengo que darle una vuelta para ver como puedo analizarlo porque aqui con todas es imposible (en el dashboard si que podria) y sino aqui escoger alguna concreta

ver tambien si meto algo aqui de numero de variaciones

marcas que dominan cada categoria de productos

In [69]:
# Contar el número de productos por marca dentro de cada categoría
marcas_por_categoria = df.groupby(["nombre_categoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")

# Encontrar la marca con más productos para cada categoría
marcas_dominantes_por_categoria = marcas_por_categoria.loc[marcas_por_categoria.groupby("nombre_categoria")["numero_productos"].idxmax()]

# Mostrar los resultados
marcas_dominantes_por_categoria

,nombre_categoria,nombre_marca,numero_productos
0,Cejas,Anastasia Beverly Hills,21
35,Desmaquillante,Collistar,5
77,Estuches maquillaje,Sephora Collection,9
127,Labios,Sephora Collection,24
168,Ojos,Sephora Collection,53
188,Paletas maquillaje,Natasha Denona,10
193,Papeles matificantes,Rem Beauty,1
194,Pestañas postizas,Huda Beauty,4
216,Pinceles & accesorios,Sephora Collection,48
219,Rizador pestañas,Dior Backstage,1


marcas que dominan cada subcategoria de productos

In [70]:
# Contar el número de productos por marca dentro de cada categoría
marcas_por_subcategoria = df.groupby(["nombre_subcategoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")

# Encontrar la marca con más productos para cada categoría
marcas_dominantes_por_subcategoria = marcas_por_subcategoria.loc[marcas_por_subcategoria.groupby("nombre_subcategoria")["numero_productos"].idxmax()]

# Mostrar los resultados
marcas_dominantes_por_subcategoria

,nombre_subcategoria,nombre_marca,numero_productos
0,Aceite desmaquillante,CHANEL,2
47,Antiojeras & corrector,Tarte,9
54,Barra de labios,Charlotte Tilbury,11
83,Barra de labios líquida,Benefit Cosmetics,3
107,Base de maquillaje,Fenty Beauty,7
142,Base de sombra de ojos,Too Faced,3
161,Brocha rostro,Sephora Collection,25
179,Bálsamo labial,Lanolips,13
198,Colorete,Benefit Cosmetics,20
241,Contouring,Sephora Collection,3
